In [1]:
import math, subprocess, time, random, os, shutil, sys
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import tsplib95 as tl
import os.path
import urllib.request
import re
import networkx as nx

## Funciones necesarias:

In [2]:
#esta funcion descarga todos los archivos que estan en "/downloads/urls.txt"
#retorna una lisa con las direcciones de cada archivo
def descargar_preinstancias():
    links = open('urls.txt', 'r')
    preinstancias = []
    for link in tqdm(links, "Descargando"):
        link = link.strip()
        name = link.rsplit('/', 1)[-1]
        filename = os.path.join('downloads', name)

        if not os.path.isfile(filename):
            try:
                urllib.request.urlretrieve(link, filename)
            except Exception as inst:
                print(inst)
                print('  Encountered unknown error. Continuing.\n URL:'+link)

        preinstancias.append(name)

    return preinstancias

In [3]:
# esta es la misma funcion dada por la catedra, modificada para devolver n y m
failed = []
def generar_instancia(archivo_entrada, archivo_salida):
    salida = open(archivo_salida,'w')
    prob = tl.load(archivo_entrada)

    grafo = prob.get_graph()
    n = grafo.number_of_nodes()
    m = grafo.number_of_edges()
    print(n,m, file=salida)
    
    for u,v in grafo.edges:
        if u == 0 or v == 0:
            os.remove(archivo_entrada)
            failed.append(archivo_entrada)
            print("El archivo"+archivo_entrada+"contiene ceros en las aristas")
            break
        if u<v:
            print(u,v,grafo.get_edge_data(u,v)['weight'], file=salida)
            
    return grafo, n , m

### Diccionario de resultados:
##### El siguiente diccionario es usado para tener los resultados de cada tipo de grafo.

In [4]:
resultados = {'a280' : 2579,'ali535' : 202310,'att48' : 10628,'att532' : 27686,'bayg29' : 1610,'bays29' : 2020,'berlin52' : 7542,'bier127' : 118282,'brazil58' : 25395,'brd14051' : 468942,'brg180' : 1950,'burma14' : 3323,'ch130' : 6110,'ch150' : 6528,'d198' : 15780,'d493' : 35002,'d657' : 48912,'d1291' : 50801,'d1655' : 62128,'d2103' : 79952,'d18512' : 644650,'dantzig42' : 699,'dsj1000' : 18659688,'eil51' : 426,'eil76' : 538,'fl417' : 11861,'fl1400' : 20127,'fl1577' : 22204,'fl3795' : 28723,'fnl4461' : 182566,'fri26' : 937,'gil262' : 2378,'gr17' : 2085,'gr21' : 2707,'gr24' : 1272,'gr48' : 5046,'gr96' : 55209,'gr120' : 6942,'gr137' : 69853,'gr202' : 40160,'gr229' : 134602,'gr431' : 171414,'gr666' : 294358,'hk48' : 11461,'kroA100' : 21282,'kroB100' : 22141,'kroC100' : 20749,'kroD100' : 21294,'kroE100' : 22068,'kroA150' : 26524,'kroB150' : 26130,'kroA200' : 29368,'kroB200' : 29437,'lin105' : 14379,'lin318' : 42029,'linhp318' : 41345,'nrw1379' : 56638,'p654' : 34643,'pa561' : 2763,'pcb442' : 50778,'pcb1173' : 56892,'pcb3038' : 137694,'pla7397' : 23260728,'pla33810' : 65913275,'pla85900' : 141904862,'pr76' : 108159,'pr107' : 44303,'pr124' : 59030,'pr136' : 96772,'pr144' : 58537,'pr152' : 73682,'pr226' : 80369,'pr264' : 49135,'pr299' : 48191,'pr439' : 107217,'pr1002' : 259045,'pr2392' : 378032,'rat99' : 1211,'rat195' : 2323,'rat575' : 6773,'rat783' : 8806,'rd100' : 7910,'rd400' : 15281,'rl1304' : 252948,'rl1323' : 270199,'rl1889' : 316536,'rl5915' : 565040,'rl5934' : 554070,'rl11849' : 920847,'si175' : 21407,'si535' : 48450,'si1032' : 92650,'st70' : 675,'swiss42' : 1273,'ts225' : 126643,'tsp225' : 3919,'u159' : 42080,'u574' : 36905,'u724' : 41910,'u1060' : 224094,'u1432' : 152970,'u1817' : 57201,'u2152' : 64253,'u2319' : 234256,'ulysses16' : 6859,'ulysses22' : 7013,'usa13509' : 19947008,'vm1084' : 239297,'vm1748' : 336556}

## Generacion de instancias:
##### Ahora usando las funciones guardaremos en el directorio "instancias" los archivos '.txt' y un 'indice.csv' para indexar toda la informacion.

In [6]:
files = descargar_preinstancias()
limite = 2000 # tamano maximo de un archivo a procesar (esto se debe a limitaciones de pc)
sin_limite = False
filas = []
for file in tqdm(files, "Archivos generados"):
    file_name = file.rsplit('.', 1)[0]
    if(int(re.search(r'\d+', file_name)[0])<limite or sin_limite):
        grafo,n,m = generar_instancia(archivo_entrada='./downloads/'+file , archivo_salida='instancias/'+file_name+'.txt')
        gral_name = file_name.rsplit('.', 1)[0]
        familia = ''.join([i for i in gral_name if not i.isdigit()])
        filas.append([file_name,familia,n,m,resultados[gral_name], 'instancias/'+file_name+'.txt'])
    
pd.DataFrame(filas, columns=["nombre", "familia","n", "m", "resultado", "path"]).to_csv("instancias/indice.csv", index=False, header=True)

Descargando: 0it [00:00, ?it/s]

Archivos generados:   0%|          | 0/96 [00:00<?, ?it/s]

El archivo./downloads/si1032.tspcontiene ceros en las aristas


In [5]:
def save_instance( instance_name, G , n):
    cant_aristas = G.number_of_edges()
    f = open(F"instancias/{instance_name}.txt", "w")
    print(n, cant_aristas, file=f)
    for line in nx.generate_edgelist(G, data=['weight']):
        linea = str(line).split(' ')
        a = int(linea[0])+1
        b = int(linea[1])+1
        c = int(linea[2])
        print(f"{a} {b} {c}", file=f)
            
            
            


In [6]:
filas_indice = []
for n in tqdm(range(10, 101),"Archivos generados:"):
    G = nx.complete_graph(n)
    for(u,v,w) in G.edges(data=True):
        w['weight'] = random.randint(1,1000)
    m = G.number_of_edges()
    save_instance( F"random-{n}", G, n)
    filas_indice.append([F"random-{n}","random", n, m,1, F"instancias/random-{n}.txt"])
result = pd.DataFrame(filas_indice, columns=["nombre", "familia","n", "m", "resultado", "path"])
df = pd.read_csv("instancias/indice.csv")
df = df[df.familia != "random"]
df = pd.concat([df, result], ignore_index = True, axis = 0)
df.to_csv("instancias/indice.csv", index=False, header=True)

Archivos generados::   0%|          | 0/91 [00:00<?, ?it/s]

In [7]:
filas_indice = []
for z in tqdm(range(0, 20),"Archivos generados:"):
    n=50
    G = nx.complete_graph(n)
    for(u,v,w) in G.edges(data=True):
        w['weight'] = random.randint(1,1000)
    m = G.number_of_edges()
    save_instance( F"random-50-{z}", G, n)
    filas_indice.append([F"random-50-{z}","random-50", n, m,1, F"instancias/random-50-{z}.txt"])
result = pd.DataFrame(filas_indice, columns=["nombre", "familia","n", "m", "resultado", "path"])
df = pd.read_csv("instancias/indice.csv")
df = df[df.familia != "random-50"]
df = pd.concat([df, result], ignore_index = True, axis = 0)
df.to_csv("instancias/indice.csv", index=False, header=True)

Archivos generados::   0%|          | 0/20 [00:00<?, ?it/s]